## GSD 


In [1]:
import os
import pandas as pd
import numpy as np
from snsynth.gsd import GSDSynthesizer
from load_data import load_data
from sklearn.model_selection import train_test_split



In [2]:
adult_path = 'adult.csv'
datasets = load_data(['adult'])
    
adult_df = datasets['adult']['data']

target =  datasets['adult']['target']
categorical_columns =  datasets['adult']['categorical_columns'].split(',')
print(adult_df.columns)
print(categorical_columns)


# Create config file. Note that we know the lower bound of each ordinal feature is 0.
# Let's assume the column upper bound is know.
ordinal_columns = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'fnlwgt']
continuous_columns = []
config = {}
for c in adult_df.columns:
    if c in categorical_columns:
        config[c] = {'type': 'string'}
    else:
        config[c] = {'type': 'int', 'lower': 0, 'upper': adult_df[c].max()}


# Split into train/test sets for machine learning evaluation.
adult_df_train, adult_df_test = train_test_split(adult_df, test_size=0.2)

loading downloaded_datasets/adult.csv
Memory consumed by adult:4167808
Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'earning-class'],
      dtype='object')
['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'earning-class']


# Adult

In [5]:
# Still need to implement 


for epsilon in [100.0]:

    N_prime = len(adult_df_train)
    
    synth = GSDSynthesizer(epsilon, 1e-5, tree_height=14, verbose=True)
    synth.fit(adult_df_train,  meta_data=config,  early_stop_threshold=0.0001, conditional_columns=['earning-class'],
              N_prime=N_prime)
        
    max_error = np.abs(synth.stat_fn(synth.data.to_numpy()) - synth.stat_fn(synth.sync_data.to_numpy())).max()
    print(f'Statistical error:', max_error)
    
    adult_sync_df = synth.sample()
    
    os.makedirs('downloaded_datasets', exist_ok=True)
    save_path = f'downloaded_datasets/adult_sync_label_{N_prime}_{epsilon:.2f}.csv'
    adult_sync_df.to_csv(save_path)
    print(f'Saving {save_path}')

privacy budgets: First moments = 2.848929. Second moments = 49.856256
Cond.Marginal= ['age'] . Sigma=0.0002. Top.Level=7. Max.Size=None
Cond.Marginal= ['fnlwgt'] . Sigma=0.0003. Top.Level=14. Max.Size=None
Cond.Marginal= ['education-num'] . Sigma=0.0002. Top.Level=5. Max.Size=None
Cond.Marginal= ['capital-gain'] . Sigma=0.0003. Top.Level=14. Max.Size=None
Cond.Marginal= ['capital-loss'] . Sigma=0.0003. Top.Level=13. Max.Size=None
Cond.Marginal= ['hours-per-week'] . Sigma=0.0002. Top.Level=7. Max.Size=None
	Total size=82484
       age.tree_height = 5. Thresholds=60
    fnlwgt.tree_height = 5. Thresholds=51
education-num.tree_height = 4. Thresholds=20
capital-gain.tree_height = 2. Thresholds=6
capital-loss.tree_height = 2. Thresholds=4
hours-per-week.tree_height = 4. Thresholds=25
Cond.Marginal= ['age', 'fnlwgt', 'earning-class'] . Sigma=0.0002. Top.Level=5. Max.Size=None
Cond.Marginal= ['age', 'education-num', 'earning-class'] . Sigma=0.0002. Top.Level=5. Max.Size=None
Cond.Marginal= ['

In [8]:
# !pip install xgboost


  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/8c/3a/c9c5d4d5c49b132ef15ac7b5ccf56ef1c82efe36cd19414771762e97c00e/xgboost-1.7.6-py3-none-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 2.1 MB/s eta 0:00:000m eta 0:00:010:00:01


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.metrics import f1_score

categorical_features = categorical_columns.copy()
numeric_features = ordinal_columns.copy()
label = 'earning-class'
categorical_features.remove(label)


numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        # ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

def sub_group(X_train_arg, y_train_arg, X_test_arg, y_test_arg, group_feature):

    clf = Pipeline(
        steps=[("preprocessor", preprocessor), ("classifier", XGBClassifier())]
    )
    
    clf.fit(X_train_arg, y_train_arg)
    
    print(f"Accuracy={clf.score(X_test_arg, y_test_arg):.4f}")
    print(f"F1-score = {f1_score(clf.predict(X_test_arg), y_test_arg, average='macro'):.4f}")
    
    group_names = X_test_arg[group_feature].unique()

    for g in group_names:
        train_sub_ids = X_train_arg[group_feature] == g
        X_train_sub = X_train_arg[train_sub_ids]
        y_train_sub = y_train_arg[train_sub_ids]
        train_sz = len(X_train_sub)

        
        test_sub_ids = X_test_arg[group_feature] == g
        X_test_sub = X_test_arg[test_sub_ids]
        y_test_sub = y_test_arg[test_sub_ids]
        test_sz = len(X_test_sub)
        print(f"{group_feature}={g}:")
        
        print(f"\tTrain: Size={train_sz:<6}, Acc-score = {clf.score(X_train_sub, y_train_sub):.4f}")

        print(f"\tTest:  Size={test_sz:<6}, Acc-score = {clf.score(X_test_sub, y_test_sub):.4f}")
        

        print(f"\tTrain: Size={train_sz:<6}, F1-score = {f1_score(y_train_sub, clf.predict(X_train_sub), average='macro'):.4f}")

        print(f"\tTest:  Size={test_sz:<6}, F1-score = {f1_score(y_test_sub, clf.predict(X_test_sub), average='macro'):.4f}")
        




In [27]:



X_real = adult_df_train[categorical_features+numeric_features]
y_real = adult_df_train[label]


X_test = adult_df_test[categorical_features+numeric_features]
y_test = adult_df_test[label]

protected_group = 'race'

print(f'Real data:')
sub_group(X_real, y_real, X_test, y_test, protected_group)




Real data:
Accuracy=0.8666
F1-score = 0.8032
race=4:
	Train: Size=22282 , Acc-score = 0.9005
	Test:  Size=5534  , Acc-score = 0.8603
	Train: Size=22282 , F1-score = 0.8631
	Test:  Size=5534  , F1-score = 0.8035
race=2:
	Train: Size=2487  , Acc-score = 0.9554
	Test:  Size=637   , Acc-score = 0.9027
	Train: Size=2487  , F1-score = 0.8860
	Test:  Size=637   , F1-score = 0.7448
race=1:
	Train: Size=826   , Acc-score = 0.9128
	Test:  Size=213   , Acc-score = 0.8451
	Train: Size=826   , F1-score = 0.8824
	Test:  Size=213   , F1-score = 0.7966
race=3:
	Train: Size=212   , Acc-score = 0.9623
	Test:  Size=59    , Acc-score = 1.0000
	Train: Size=212   , F1-score = 0.8986
	Test:  Size=59    , F1-score = 1.0000
race=0:
	Train: Size=241   , Acc-score = 0.9378
	Test:  Size=70    , Acc-score = 0.9857
	Train: Size=241   , F1-score = 0.8355
	Test:  Size=70    , F1-score = 0.9507


In [28]:

data_path = 'downloaded_datasets/adult_sync_26048_100.00.csv'
adult_sync_df = pd.read_csv(data_path, index_col=0)

print(data_path)
X_sync = adult_sync_df[categorical_features+numeric_features]
y_sync = adult_sync_df[label]
print(f'Sync data:')
sub_group(X_sync, y_sync, X_test, y_test, protected_group)

downloaded_datasets/adult_sync_26048_100.00.csv
Sync data:
Accuracy=0.8465
F1-score = 0.7729
race=4:
	Train: Size=22282 , Acc-score = 0.8929
	Test:  Size=5534  , Acc-score = 0.8388
	Train: Size=22282 , F1-score = 0.8515
	Test:  Size=5534  , F1-score = 0.7727
race=2:
	Train: Size=2486  , Acc-score = 0.9413
	Test:  Size=637   , Acc-score = 0.8901
	Train: Size=2486  , F1-score = 0.8411
	Test:  Size=637   , F1-score = 0.7042
race=1:
	Train: Size=827   , Acc-score = 0.9178
	Test:  Size=213   , Acc-score = 0.8357
	Train: Size=827   , F1-score = 0.8905
	Test:  Size=213   , F1-score = 0.7868
race=3:
	Train: Size=212   , Acc-score = 0.9764
	Test:  Size=59    , Acc-score = 0.9661
	Train: Size=212   , F1-score = 0.9353
	Test:  Size=59    , F1-score = 0.4914
race=0:
	Train: Size=241   , Acc-score = 0.9419
	Test:  Size=70    , Acc-score = 0.9857
	Train: Size=241   , F1-score = 0.8380
	Test:  Size=70    , F1-score = 0.9507


# End